In [1]:
import pandas as pd
M = pd.read_pickle('../data/given/M.pkl')
N = None #pd.read_pickle('../data/given/N.pkl')
#test = pd.read_pickle('../data/given/test.pkl')

In [2]:
from common import *

In [3]:
pd.set_option('display.max_rows', 2000)

In [4]:
def setup_P(*,P):
    ##P.rename(columns={'returnsOpenNextMktres10':'y'}, inplace=True)
    P['y'] = P.returnsOpenNextMktres10
    #globalStd = (P.universe*P.y).std()
    #grp = P.groupby(F.assetCodeId)[['y','universe']]
    #assetStds = grp.apply(lambda x: np.sqrt((x.universe*x.y**2).sum()/(x.universe).sum()) if x.universe.sum()>25 else globalStd)
    #P['assetCodeId'] = P.assetCode.map(assetCodeIdAssign)
    #P.y /= P.assetCodeId.map(assetStds)
    P['target'] = P.y>0
    P['upDown'] = (P.target*2-1)
    P['upDown1'] = P.upDown*P.universe.astype(int)
    P['absVal'] = np.abs(P.y)
    P['absVal1'] = P.absVal*P.universe
    P['weight'] = P.absVal#.qtl()
    P['weight1'] = P.weight*P.universe

## feature sets definitions

In [11]:
fThefault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","cc0","oo0","cc_10","oo_10","cc0_10","oo0_10","dayOfYear","dayOfWeek","af","it","ooSince21","ooMaxSince21","ooDrawdownSince21","ooMinSince21","ooDrawupSince21","ooMaxDrawup5Since21","ooMaxDrawup10Since21","ooMaxDrawup(10-5)Since21","ooMaxDrawup(20-10)Since21","ooSince62","ooMaxSince62","ooDrawdownSince62","ooMinSince62","ooDrawupSince62","ooMaxDrawup5Since62","ooMaxDrawup10Since62","ooMaxDrawup(10-5)Since62","ooMaxDrawup(20-10)Since62","ooSince125","ooMaxSince125","ooDrawdownSince125","ooMinSince125","ooDrawupSince125","ooMaxDrawup5Since125","ooMaxDrawup10Since125","ooMaxDrawup(10-5)Since125","ooMaxDrawup(20-10)Since125","ooSince250","ooMaxSince250","ooDrawdownSince250","ooMinSince250","ooDrawupSince250","ooMaxDrawup5Since250","ooMaxDrawup10Since250","ooMaxDrawup(10-5)Since250","ooMaxDrawup(20-10)Since250","oo0Since21","oo0MaxSince21","oo0DrawdownSince21","oo0MinSince21","oo0DrawupSince21","oo0MaxDrawup5Since21","oo0MaxDrawup10Since21","oo0MaxDrawup(10-5)Since21","oo0MaxDrawup(20-10)Since21","oo0Since62","oo0MaxSince62","oo0DrawdownSince62","oo0MinSince62","oo0DrawupSince62","oo0MaxDrawup5Since62","oo0MaxDrawup10Since62","oo0MaxDrawup(10-5)Since62","oo0MaxDrawup(20-10)Since62","oo0Since125","oo0MaxSince125","oo0DrawdownSince125","oo0MinSince125","oo0DrawupSince125","oo0MaxDrawup5Since125","oo0MaxDrawup10Since125","oo0MaxDrawup(10-5)Since125","oo0MaxDrawup(20-10)Since125","oo0Since250","oo0MaxSince250","oo0DrawdownSince250","oo0MinSince250","oo0DrawupSince250","oo0MaxDrawup5Since250","oo0MaxDrawup10Since250","oo0MaxDrawup(10-5)Since250","oo0MaxDrawup(20-10)Since250","oo_{(0..4)^1}","oo_{(0..9)^1}","oo_{(0..14)^1}","oo_{(0..19)^1}","oo_{(5..9)^1}","oo_{(5..14)^1}","oo_{(5..19)^1}","oo_{(10..14)^1}","oo_{(10..19)^1}","oo_{(15..19)^1}","oo0_{(0..4)^1}","oo0_{(0..9)^1}","oo0_{(0..14)^1}","oo0_{(0..19)^1}","oo0_{(5..9)^1}","oo0_{(5..14)^1}","oo0_{(5..19)^1}","oo0_{(10..14)^1}","oo0_{(10..19)^1}","oo0_{(15..19)^1}","(oo-oo0)_{(0..4)^1}","(oo-oo0)_{(0..9)^1}","(oo-oo0)_{(0..14)^1}","(oo-oo0)_{(0..19)^1}","(oo-oo0)_{(5..9)^1}","(oo-oo0)_{(5..14)^1}","(oo-oo0)_{(5..19)^1}","(oo-oo0)_{(10..14)^1}","(oo-oo0)_{(10..19)^1}","(oo-oo0)_{(15..19)^1}","(oo-oo0){0/1}","(oo-oo0){0/2}","(oo-oo0){1/2}","(oo-oo0){0}","(oo-oo0){1}","(oo-oo0){2}","(cc-cc0){0/1}","(cc-cc0){0/2}","(cc-cc0){1/2}","(cc-cc0){0}","(cc-cc0){1}","(cc-cc0){2}","af{1}","it{1}","(it,af,it{1},oo0,cc0){0/1}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/1}","(it,af,it{1},oo0,cc0){0/2}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/2}","(it,af,it{1},oo0,cc0){0/3}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/3}","(it,af,it{1},oo0,cc0){0/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){0/4}","(it,af,it{1},oo0,cc0){1/2}","(it{1},af{1},it{2},oo0{1},cc0{1}){1/2}","(it,af,it{1},oo0,cc0){1/3}","(it{1},af{1},it{2},oo0{1},cc0{1}){1/3}","(it,af,it{1},oo0,cc0){1/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){1/4}","(it,af,it{1},oo0,cc0){2/3}","(it{1},af{1},it{2},oo0{1},cc0{1}){2/3}","(it,af,it{1},oo0,cc0){2/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){2/4}","(it,af,it{1},oo0,cc0){3/4}","(it{1},af{1},it{2},oo0{1},cc0{1}){3/4}","volumeRatioMeanBack2","volumeRatioMeanBack2{1}","volumeRatioMean5","volumeRatioMean10","volumeRatioMean10{5}","volumeRatioMean5ByMean20","oo0_20","oo0_60","oo0_120","oo0_250","ooMaxDrawdown5Since21","ooMaxDrawdown10Since21","ooMaxDrawdown(10-5)Since21","ooMaxDrawdown(20-10)Since21","ooPrev5MaxSince21","ooPrev10MaxSince21","ooPrev5MinSince21","ooPrev10MinSince21","ooMaxDrawdown5Since62","ooMaxDrawdown10Since62","ooMaxDrawdown(10-5)Since62","ooMaxDrawdown(20-10)Since62","ooPrev5MaxSince62","ooPrev10MaxSince62","ooPrev5MinSince62","ooPrev10MinSince62","ooMaxDrawdown5Since125","ooMaxDrawdown10Since125","ooMaxDrawdown(10-5)Since125","ooMaxDrawdown(20-10)Since125","ooPrev5MaxSince125","ooPrev10MaxSince125","ooPrev5MinSince125","ooPrev10MinSince125","ooMaxDrawdown5Since250","ooMaxDrawdown10Since250","ooMaxDrawdown(10-5)Since250","ooMaxDrawdown(20-10)Since250","ooPrev5MaxSince250","ooPrev10MaxSince250","ooPrev5MinSince250","ooPrev10MinSince250","oo0MaxDrawdown5Since21","oo0MaxDrawdown10Since21","oo0MaxDrawdown(10-5)Since21","oo0MaxDrawdown(20-10)Since21","oo0Prev5MaxSince21","oo0Prev10MaxSince21","oo0Prev5MinSince21","oo0Prev10MinSince21","oo0MaxDrawdown5Since62","oo0MaxDrawdown10Since62","oo0MaxDrawdown(10-5)Since62","oo0MaxDrawdown(20-10)Since62","oo0Prev5MaxSince62","oo0Prev10MaxSince62","oo0Prev5MinSince62","oo0Prev10MinSince62","oo0MaxDrawdown5Since125","oo0MaxDrawdown10Since125","oo0MaxDrawdown(10-5)Since125","oo0MaxDrawdown(20-10)Since125","oo0Prev5MaxSince125","oo0Prev10MaxSince125","oo0Prev5MinSince125","oo0Prev10MinSince125","oo0MaxDrawdown5Since250","oo0MaxDrawdown10Since250","oo0MaxDrawdown(10-5)Since250","oo0MaxDrawdown(20-10)Since250","oo0Prev5MaxSince250","oo0Prev10MaxSince250","oo0Prev5MinSince250","oo0Prev10MinSince250"]
print('fThefault')

fThefault


In [12]:
fNewfault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","cc0","oo0","cc_10","oo_10","cc0_10","oo0_10","dayOfYear","dayOfWeek","ooSince21","ooMaxSince21","ooDrawdownSince21","ooMaxDrawdown5Since21","ooMaxDrawdown10Since21","ooPrev10MaxSince21","ooMinSince21","ooPrev10MinSince21","ooSince62","ooMaxSince62","ooDrawdownSince62","ooMaxDrawdown5Since62","ooMaxDrawdown10Since62","ooPrev10MaxSince62","ooMinSince62","ooPrev10MinSince62","ooSince125","ooMaxSince125","ooDrawdownSince125","ooMaxDrawdown5Since125","ooMaxDrawdown10Since125","ooPrev10MaxSince125","ooMinSince125","ooPrev10MinSince125","ooSince250","ooMaxSince250","ooDrawdownSince250","ooMaxDrawdown5Since250","ooMaxDrawdown10Since250","ooPrev10MaxSince250","ooMinSince250","ooPrev10MinSince250","oo0Since21","oo0MaxSince21","oo0DrawdownSince21","oo0MaxDrawdown5Since21","oo0MaxDrawdown10Since21","oo0Prev10MaxSince21","oo0MinSince21","oo0Prev10MinSince21","oo0Since62","oo0MaxSince62","oo0DrawdownSince62","oo0MaxDrawdown5Since62","oo0MaxDrawdown10Since62","oo0Prev10MaxSince62","oo0MinSince62","oo0Prev10MinSince62","oo0Since125","oo0MaxSince125","oo0DrawdownSince125","oo0MaxDrawdown5Since125","oo0MaxDrawdown10Since125","oo0Prev10MaxSince125","oo0MinSince125","oo0Prev10MinSince125","oo0Since250","oo0MaxSince250","oo0DrawdownSince250","oo0MaxDrawdown5Since250","oo0MaxDrawdown10Since250","oo0Prev10MaxSince250","oo0MinSince250","oo0Prev10MinSince250","oo_{(0..4)^1}","oo_{(0..9)^1}","oo_{(0..14)^1}","oo_{(0..19)^1}","oo_{(5..9)^1}","oo_{(5..14)^1}","oo_{(5..19)^1}","oo_{(10..14)^1}","oo_{(10..19)^1}","oo_{(15..19)^1}","oo0_{(0..4)^1}","oo0_{(0..9)^1}","oo0_{(0..14)^1}","oo0_{(0..19)^1}","oo0_{(5..9)^1}","oo0_{(5..14)^1}","oo0_{(5..19)^1}","oo0_{(10..14)^1}","oo0_{(10..19)^1}","oo0_{(15..19)^1}","(it,af,it{1},oo0,cc0){3/4}"]
fNewnewfault = fNewfault[:-1] + ['(oo0-cc0)']
print('fNewfault')

fNewfault


In [13]:
f2firstkaggle = ["assetCodeId","volume","close","open","assetNameId","cc","oo","cc0","oo0","cc_10","oo_10","cc0_10","oo0_10","dayOfYear","dayOfWeek","af","it","ooSince21","ooMaxSince21","ooDrawdownSince21","ooMinSince21","ooDrawupSince21","ooMaxDrawup10Since21","ooMaxDrawup(20-10)Since21","ooSince125","ooMaxSince125","ooDrawdownSince125","ooMinSince125","ooDrawupSince125","ooMaxDrawup10Since125","ooMaxDrawup(20-10)Since125","oo_{(0..4)^1}","oo_{(0..9)^1}","oo_{(0..14)^1}","oo_{(0..19)^1}","oo_{(5..9)^1}","oo_{(5..14)^1}","oo_{(5..19)^1}","oo_{(10..14)^1}","oo_{(10..19)^1}","oo_{(15..19)^1}","oo0_{(0..4)^1}","oo0_{(0..9)^1}","oo0_{(0..14)^1}","oo0_{(0..19)^1}","oo0_{(5..9)^1}","oo0_{(5..14)^1}","oo0_{(5..19)^1}","oo0_{(10..14)^1}","oo0_{(10..19)^1}","oo0_{(15..19)^1}","(oo-oo0)_{(0..4)^1}","(oo-oo0)_{(0..9)^1}","(oo-oo0)_{(0..14)^1}","(oo-oo0)_{(0..19)^1}","(oo-oo0)_{(5..9)^1}","(oo-oo0)_{(5..14)^1}","(oo-oo0)_{(5..19)^1}","(oo-oo0)_{(10..14)^1}","(oo-oo0)_{(10..19)^1}","(oo-oo0)_{(15..19)^1}","(it,af,it{1},oo0,cc0){0/1}","(it,af,it{1},oo0,cc0){0/2}","(it,af,it{1},oo0,cc0){0/3}","(it,af,it{1},oo0,cc0){0/4}","(it,af,it{1},oo0,cc0){1/2}","(it,af,it{1},oo0,cc0){1/3}","(it,af,it{1},oo0,cc0){1/4}","(it,af,it{1},oo0,cc0){2/3}","(it,af,it{1},oo0,cc0){2/4}","(it,af,it{1},oo0,cc0){3/4}","volumeRatioMeanBack2","volumeRatioMeanBack2{1}","volumeRatioMean5","volumeRatioMean10","oo0_20","oo0_120","oo_20","oo_120","(oo-oo0)_10","(oo-oo0)_20","(oo-oo0)_120","ooMaxDrawdown10Since21","ooMaxDrawdown(20-10)Since21","ooPrev10MaxSince21","ooPrev10MinSince21","ooMaxDrawdown10Since125","ooMaxDrawdown(20-10)Since125","ooPrev10MaxSince125","ooPrev10MinSince125"]#,"assetCodeIdWTFlogabs","assetCodeIdWTFsin2","assetCodeIdWTFFlogabs","assetCodeIdWTFFsin2"]
print('f2firstkaggle')

f2firstkaggle


In [14]:
fTime = 'dayOfYear dayOfWeek'.split()
fAsset = 'assetCodeId assetNameId'.split()
fPriceAbsolute = 'close open'.split()
print('fTime fAsset fPriceAbsolute')

fTime fAsset fPriceAbsolute


In [15]:
def select_relative(f):
    return f not in set(fTime+fAsset+fPriceAbsolute)

fThefaultRelative = list(filter(select_relative, fThefault))

fHack1 = ['assetCodeIdWTFlogabs', 'assetCodeIdWTFsin2']
fHack2 = ['assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']
fNewfaultRelativeHack0 = list(filter(select_relative, fNewfault)) + fHack2

print('select_relative combinations... |')

select_relative combinations... |


##### choose our feature selection:

In [16]:
fSelection = list(filter(select_relative, fNewnewfault)) + ['assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']

## load the data frames to compare

In [112]:
#F,P = pd.read_pickle('/big/data/saves/train_6withglobal.32.pkl')

In [ ]:
Fvalues = np.load('/big/data/saves/train_6withglobal.32.npy')
Findex,Fcolumns,P = pickle.loads(Path('/big/data/saves/train_6withglobal.32.side').read_bytes())
F = pd.DataFrame(Fvalues, index=Findex, columns=Fcolumns, copy=False)

#### need to make sure feature generation is bug-free

In [16]:
vct = F.nunique()

In [53]:
# reference code to examine column
c = '(oo-aoo)10'
pd.concat([F[c].groupby(P.time).first()], axis=1)
F.m_oo

732812     0.0
732814     0.0
732815     0.0
732816     0.0
732817     0.0
732818     0.0
732819     0.0
732820     0.0
732821     0.0
732822     0.0
732823     0.0
732824     0.0
732825     0.0
732826     0.0
732827     0.0
732828     0.0
732829     0.0
732830     0.0
732831     0.0
732832     0.0
732833     0.0
732834     0.0
732835     0.0
732836     0.0
732837     0.0
732838     0.0
732839     0.0
732840     0.0
732841     0.0
732842     0.0
732843     0.0
732844     0.0
732845     0.0
732848     0.0
732849     0.0
732850     0.0
732851     0.0
732852     0.0
732854     0.0
732855     0.0
732856     0.0
732857     0.0
732858     0.0
732859     0.0
732860     0.0
732863     0.0
732866     0.0
732867     0.0
732868     0.0
732869     0.0
732870     0.0
732871     0.0
732872     0.0
732874     0.0
732875     0.0
732876     0.0
732877     0.0
732878     0.0
732881     0.0
732882     0.0
732883     0.0
732884     0.0
732885     0.0
732886     0.0
732888     0.0
732889     0.0
732890    

In [24]:
F[F.assetCodeId==3].oo1MinSince62

732815    -0.238038
734404    -0.115435
735994    -0.203804
737587    -0.047878
739182    -0.131608
740777    -0.044383
742373    -0.245605
743970    -0.350507
745564    -0.236402
747159    -0.214711
748755    -0.211197
750350    -0.215812
751947    -0.198645
753546    -0.201734
755147    -0.194304
756746    -0.126494
758346    -0.178856
759950    -0.182007
761554    -0.236786
763160    -0.310071
764762    -0.300524
766369    -0.286833
767980    -0.324632
769592    -0.315591
771204    -0.239858
772817    -0.220877
774437    -0.232409
776059    -0.176329
777679    -0.151452
779300    -0.124053
780924    -0.163594
782552    -0.107246
784185    -0.119598
785824    -0.120491
787464    -0.067717
789108    -0.030596
790754    -0.068070
792396    -0.174112
794038    -0.125163
795682    -0.174957
797332    -0.138722
798984    -0.124053
800637    -0.116916
802290    -0.171360
803942    -0.127380
805594    -0.200396
807247    -0.208802
808898    -0.207474
810548    -0.200499
812198    -0.155659


In [1]:
vct

NameError: name 'vct' is not defined

In [150]:
BUG.V_.m_oo[-5:]

array([ 0.01469621,  0.01152872, -0.00175254, -0.01747028,  0.00641909])

#### more detailed comparison

start load

In [4]:
F0, P0 = pd.read_pickle('/big/data/saves/train_4Newnewfault64xx.pkl')

end load

In [17]:
set(F0.columns)^set(fSelection)

{'assetCodeId',
 'assetCodeIdWTFlogabs',
 'assetCodeIdWTFsin2',
 'assetNameId',
 'close',
 'dayOfWeek',
 'dayOfYear',
 'open'}

In [65]:
F0['(oo0-cc0)'] = F0['(it,af,it{1},oo0,cc0){3/4}']

In [66]:
F0.drop(columns=[c for c in F0.columns if c not in fNewnewfault], inplace=True)

In [69]:
gc.collect()
df = F[F0.columns]
df0 = F0
eq = np.abs(df0.fillna(np.inf)-df.fillna(np.inf)).fillna(0)<1e-8

In [70]:
eq.sum(axis=0)/len(eq)

volume                      1.000000
close                       1.000000
open                        1.000000
assetCodeId                 1.000000
assetNameId                 1.000000
cc                          1.000000
oo                          1.000000
cc0                         1.000000
oo0                         1.000000
cc_10                       1.000000
oo_10                       1.000000
cc0_10                      1.000000
oo0_10                      1.000000
dayOfYear                   1.000000
dayOfWeek                   1.000000
ooSince21                   1.000000
ooMaxSince21                1.000000
ooDrawdownSince21           1.000000
ooMinSince21                1.000000
ooSince62                   1.000000
ooMaxSince62                1.000000
ooDrawdownSince62           1.000000
ooMinSince62                1.000000
ooSince125                  1.000000
ooMaxSince125               1.000000
ooDrawdownSince125          1.000000
ooMinSince125               1.000000
o

In [57]:
dude = M[M.assetCode=='AAPL.O'][M.time<'2009-01-03'].tail(5)
set_basic_features(M=dude)
dude

,time,assetCode,assetName,volume,close,open,returnsOpenNextMktres10,universe,assetCodeId,assetNameId,cc,oo,cc0,oo0,cc_10,oo_10,cc0_10,oo0_10,dayOfYear,dayOfWeek
726443,2008-12-26 22:00:00+00:00,AAPL.O,Apple Inc,11043071.0,85.81,86.55,0.023792,1.0,3,3,0.009014,0.005213,0.005038,0.001207,-0.101741,-0.115534,-0.100484,-0.078656,361.0,4.0
728035,2008-12-29 22:00:00+00:00,AAPL.O,Apple Inc,24567225.0,86.61,86.62,-0.008313,1.0,3,3,0.009280,0.000808,0.011972,-0.001201,-0.126304,-0.069885,-0.109256,-0.070672,364.0,0.0
729627,2008-12-30 22:00:00+00:00,AAPL.O,Apple Inc,34579620.0,86.29,87.42,0.008330,1.0,3,3,-0.003702,0.009193,-0.020864,0.010283,-0.093528,-0.093416,-0.130348,-0.078830,365.0,1.0
731221,2008-12-31 22:00:00+00:00,AAPL.O,Apple Inc,21715763.0,85.35,86.00,0.052227,1.0,3,3,-0.010953,-0.016377,-0.023658,-0.028226,-0.111633,-0.088947,-0.093581,-0.116575,366.0,2.0
732815,2009-01-02 22:00:00+00:00,AAPL.O,Apple Inc,26964210.0,90.75,85.58,-0.026166,1.0,3,3,0.061348,-0.004896,0.032733,-0.015290,0.017676,-0.061408,-0.029549,-0.053305,2.0,4.0


In [58]:
dude.oo.sum()

-0.0060578004450126009

In [63]:
c = '(oo0-cc0)'
guy = pd.concat({'time':P.time,'assetCodeId':F.assetCodeId,'df':df[c],'df0': df0[c],'cc0':df.cc0,'oo0':df.oo0}, axis=1)
guy[(guy.assetCodeId!=-1) & (df[c]!=df0[c])].pipe(lambda x: pd.concat([x, x.oo0-x.cc0], axis=1))

,assetCodeId,cc0,df,df0,oo0,time,0
732812,0,0.028696,1.256251e-02,-0.064784,4.125897e-02,2009-01-02 22:00:00+00:00,1.256251e-02
732813,1,-0.018934,-3.017972e-02,0.042388,-4.911375e-02,2009-01-02 22:00:00+00:00,-3.017972e-02
732814,2,-0.010750,1.986397e-02,0.021261,9.114077e-03,2009-01-02 22:00:00+00:00,1.986397e-02
732815,3,0.032733,-4.802233e-02,-0.004568,-1.528981e-02,2009-01-02 22:00:00+00:00,-4.802233e-02
732816,1564,-0.055761,1.097132e-01,0.002326,5.395219e-02,2009-01-02 22:00:00+00:00,1.097132e-01
732817,4,-0.017254,3.152896e-02,-0.011766,1.427474e-02,2009-01-02 22:00:00+00:00,3.152896e-02
732818,5,-0.017041,3.983991e-02,0.004516,2.279893e-02,2009-01-02 22:00:00+00:00,3.983991e-02
732819,7,-0.045668,4.680588e-02,-0.029059,1.138005e-03,2009-01-02 22:00:00+00:00,4.680588e-02
732820,2099,-0.042891,9.170951e-02,-0.053478,4.881807e-02,2009-01-02 22:00:00+00:00,9.170951e-02
732821,8,-0.008815,7.970818e-03,0.010375,-8.440820e-04,2009-01-02 22:00:00+00:00,7.970818e-03


In [58]:
for c in combos.omo(5):
    name = '(it,af,it{{1}},oo0,cc0){{{}}}'.format(combos.name.index(c))
    if name == '(it,af,it{1},oo0,cc0){3/4}':
        print(c)
        print(c@np.asarray([[0]*9,[1]*9,[2]*9,[4]*9,[8.]*9]))

[ 0  0  0  1 -1]
[-4. -4. -4. -4. -4. -4. -4. -4. -4.]
